<a href="https://colab.research.google.com/github/valerio-unifei/UNIFEI-IA-Aulas/blob/main/UNIFEI_IA_Sistema_Especialista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando Pacote

In [ ]:
!pip install experta

  Created wheel for frozendict: filename=frozendict-1.2-cp37-none-any.whl size=3150 sha256=1a6e137bd0b2f1a44082c1b54e992a3ecc88e0a8d58c7e75a37f707f31d39d90
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built frozendict


In [ ]:
from experta import *

# Exemplos

## Salvando o Dia

In [ ]:
# Classe para definir as variáveis dos Fatos
class Meta(Fact):
    pass

class Heroi(Fact):
    nome = Field(str)
    status = Field(str)

# máquina de inferência com a base de conhecimento
class MI(KnowledgeEngine):
    @DefFacts()
    def goal_and_heroes(self):
        yield Meta('salvar-o-dia')
        yield Heroi(nome="Superhomi", status="desocupado")
        yield Heroi(nome="Homi-aranho", status="desocupado")
        yield Heroi(nome="Batima", status="desocupado")
    @Rule(
        Meta('salvar-o-dia'),
        EXISTS(
            Heroi(status="desocupado")
        )
    )
    def salvar_o_dia(self):
        print("O dia foi salvo")

In [ ]:
# criando máquina de inferência
mi = MI()
mi.reset()
# vendo os objetivos
mi.agenda

0: salvar_o_dia {Meta('salvar-o-dia'), InitialFact()}

In [ ]:
mi.facts

FactList([(0, InitialFact()),
          (1, Meta('salvar-o-dia')),
          (2, Heroi(nome='Superhomi', status='desocupado')),
          (3, Heroi(nome='Homi-aranho', status='desocupado')),
          (4, Heroi(nome='Batima', status='desocupado'))])

In [ ]:
mi.run()

O dia foi salvo


## Cálculo Fatorial

In [ ]:
class Fatorial(Fact):
    n = Field(lambda n: isinstance(n, int) and n >= 0, mandatory=True)
    resultado = Field(int, mandatory=True)

# máquina de inferencia Fatorial
class MIF(KnowledgeEngine):
    @DefFacts()
    def first(self):
        yield Fatorial(n=0, resultado=1)

    @Rule(
        AS.f << Fatorial(
            n=MATCH.n,
            resultado=MATCH.r))
    def fatorial(self, f, n, r):
        self.declare(
            Fatorial(
                n=n + 1,
                resultado=(n + 1) * r))
        self.retract(f)

In [ ]:
mf = MIF()
mf.reset()
mf.run(1000)
mf.facts

FactList([(0, InitialFact()),
          (1001,
           Fatorial(n=1000, resultado=402387260077093773543702433923003985719374864210714632543799910429938512398629020592044208486969404800479988610197196058631666872994808558901323829669944590997424504087073759918823627727188732519779505950995276120874975462497043601418278094646496291056393887437886487337119181045825783647849977012476632889835955735432513185323958463075557409114262417474349347553428646576611667797396668820291207379143853719588249808126867838374559731746136085379534524221586593201928090878297308431392844403281231558611036976801357304216168747609675871348312025478589320767169132448426236131412508780208000261683151027341827977704784635868170164365024153691398281264810213092761244896359928705114964975419909342221566832572080821333186116811553615836546984046708975602900950537616475847728421889679646244945160765353408198901385442487984959953319101723355556602139450399736280750137837615307127761926849034352625200015888535147331

## Problema de Macacos e Bananas

O objetivo é que o macaco encontre e coma algumas bananas.

In [ ]:
import schema

class Monkey(Fact):
    location = Field(str, default="green-couch")
    on_top_of = Field(str, default="floor")
    holding = Field(str, default="blank")

    
class Thing(Fact):
    name = Field(str, mandatory=True)
    location = Field(str, mandatory=True)
    on_top_of = Field(str, default="floor")
    weight = Field(schema.Or("light", "heavy"), default="light")

class Chest(Fact):
    name = Field(str, mandatory=True)
    contents = Field(str, mandatory=True)
    unlocked_by = Field(str, mandatory=True)


class GoalIsTo(Fact):
    action = Field(schema.Or("hold", "unlock", "eat", "move", "on", "walk-to"),
                   mandatory=True)
    arguments = Field(schema.Or(str, [str]), mandatory=True)
class ChestUnlockingRules:
    """CHEST UNLOCKING RULES"""

    @Rule(
         GoalIsTo(action="unlock", arguments=MATCH.chest),
         Thing(name=MATCH.chest, on_top_of=NE("floor"), weight="light"),
         Monkey(holding=~MATCH.chest),
         NOT(GoalIsTo(action="hold", arguments=MATCH.chest)))
    def hold_chest_to_put_on_floor(self, chest):
        self.declare(GoalIsTo(action="hold", arguments=chest))

    @Rule(
        GoalIsTo(
            action="unlock",
            arguments=MATCH.chest),
        AS.monkey << Monkey(
                         location=MATCH.place,
                         on_top_of=MATCH.on,
                         holding=MATCH.chest),
        AS.thing  << Thing(
                         name=MATCH.chest))
    def put_chest_on_floor(self, monkey, thing, place, on, chest):
        print("Monkey throws the %s off the %s onto the floor." % (chest, on))
        self.modify(monkey, holding="blank")
        self.modify(thing, location=place, on_top_of="floor")

    @Rule(
        GoalIsTo(action="unlock", arguments=MATCH.obj),
        Thing(name=MATCH.obj, on_top_of="floor"),
        Chest(name=MATCH.obj, unlocked_by=MATCH.key),
        Monkey(holding=~MATCH.key),
        NOT(GoalIsTo(action="hold", arguments=MATCH.key)))
    def get_key_to_unlock(self, key):
        self.declare(GoalIsTo(action="hold", arguments=key))

    @Rule(
        GoalIsTo(
            action="unlock",
            arguments=MATCH.chest),
        Monkey(
            location=MATCH.mplace,
            holding=MATCH.key),
        Thing(
            name=MATCH.chest,
            location=MATCH.cplace & ~MATCH.mplace,
            on_top_of="floor"),
        Chest(
            name=MATCH.chest,
            unlocked_by=MATCH.key),
        NOT(
            GoalIsTo(
                action="walk-to",
                arguments=MATCH.cplace)))
    def move_to_chest_with_key(self, cplace):
        self.declare(GoalIsTo(action="walk-to", arguments=cplace))
    
    @Rule(
        AS.goal << GoalIsTo(action="unlock", arguments=MATCH.name),
        AS.chest << Chest(name=MATCH.name, contents=MATCH.contents, unlocked_by=MATCH.key),
        Thing(name=MATCH.name, location=MATCH.place, on_top_of=MATCH.on),
        Monkey(location=MATCH.place, on_top_of=MATCH.on, holding=MATCH.key))
    def unlock_chest_with_key(self, goal, name, key, contents, place, chest):
        print("Monkey opens the %s with the %s revealing the %s." % (name, key, contents))
        self.modify(chest, contents="nothing")
        self.declare(Thing(name=contents, location=place, on_top_of=name, weight="light"))
        self.retract(goal)

class HoldObjectRules:
    """HOLD OBJECT RULES"""

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Chest(name=MATCH.chest, contents=MATCH.obj),
        NOT(GoalIsTo(action="unlock", arguments=MATCH.chest)))
    def unlock_chest_to_hold_object(self, chest):
        self.declare(GoalIsTo(action="unlock", arguments=chest))

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of="ceiling", weight="light"),
        NOT(Thing(name="ladder", location=MATCH.place)),
        NOT(GoalIsTo(action="move", arguments__0="ladder", arguments__1=MATCH.place)))
    def use_ladder_to_hold(self, place):
        self.declare(GoalIsTo(action="move", arguments=["ladder", place]))

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of="ceiling", weight="light"),
        Thing(name="ladder", location=MATCH.place, on_top_of="floor"),
        Monkey(on_top_of=NE("ladder")),
        NOT(GoalIsTo(action="on", arguments="ladder")))
    def climb_ladder_to_hold(self):
        self.declare(GoalIsTo(action="on", arguments="ladder"))

    @Rule(
        AS.goal << GoalIsTo(action="hold", arguments=MATCH.name),
        AS.thing << Thing(name=MATCH.name, location=MATCH.place, on_top_of="ceiling", weight="light"),
        Thing(name="ladder", location=MATCH.place),
        AS.monkey << Monkey(location=MATCH.place, on_top_of="ladder", holding="blank"))
    def grab_object_from_ladder(self, thing, monkey, goal, name):
        print("Monkey grabs the %s." % name)
        self.modify(thing, location="held", on_top_of="held")
        self.modify(monkey, holding=name)
        self.retract(goal)

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of=MATCH.on & NE("ceiling"), weight="light"),
        Monkey(location=MATCH.place, on_top_of=~MATCH.on),
        NOT(GoalIsTo(action="on", arguments=MATCH.on)))
    def climb_to_hold(self, on):
        self.declare(GoalIsTo(action="on", arguments=on))

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of=NE("ceiling"), weight="light"),
        Monkey(location=~MATCH.place),
        NOT(GoalIsTo(action="walk-to", arguments=MATCH.place)))
    def walk_to_hold(self, place):
        self.declare(GoalIsTo(action="walk-to", arguments=place))

    @Rule(
        GoalIsTo(action="hold", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of=MATCH.on, weight="light"),
        Monkey(location=MATCH.place, on_top_of=MATCH.on, holding=NE("blank")),
        NOT(GoalIsTo(action="hold", arguments="blank")))
    def drop_to_hold(self):
        self.declare(GoalIsTo(action="hold", arguments="blank"))

    @Rule(
      AS.goal << GoalIsTo(action="hold", arguments=MATCH.name),
      AS.thing << Thing(name=MATCH.name, location=MATCH.place, on_top_of=MATCH.on, weight="light"),
      AS.monkey << Monkey(location=MATCH.place, on_top_of=MATCH.on, holding="blank"))
    def grab_object(self, name, monkey, thing, goal):
        print("Monkey grabs the %s." % name)
        self.modify(thing, location="held", on_top_of="held")
        self.modify(monkey, holding=name)
        self.retract(goal)

    @Rule(
        AS.goal << GoalIsTo(action="hold", arguments="blank"),
        AS.monkey << Monkey(location=MATCH.place, on_top_of=MATCH.on, holding=MATCH.name & NE("blank")),
        AS.thing << Thing(name=MATCH.name))
    def drop_object(self, monkey, thing, place, name, on, goal):
        print("Monkey drops the %s." % name)
        self.modify(monkey, holding="blank")
        self.modify(thing, location=place, on_top_of=on)
        self.retract(goal)

class MoveObjectRules:
    """MOVE OBJECT RULES"""

    @Rule(
        GoalIsTo(action="move", arguments__0=MATCH.obj),
        Chest(name=MATCH.chest, contents=MATCH.obj),
        NOT(GoalIsTo(action="unlock", arguments=MATCH.chest)))
    def unlock_chest_to_move_object(self, chest):
        self.declare(GoalIsTo(action="unlock", arguments=chest))

    @Rule(
        GoalIsTo(action="move", arguments__0=MATCH.obj, arguments__1=MATCH.place),
        Thing(name=MATCH.obj, location=~MATCH.place, weight="light"),
        Monkey(holding=~MATCH.obj),
        NOT(GoalIsTo(action="hold", arguments=MATCH.obj)))
    def hold_object_to_move(self, obj):
        self.declare(GoalIsTo(action="hold", arguments=obj))

    @Rule(
        GoalIsTo(action="move", arguments__0=MATCH.obj, arguments__1=MATCH.place),
        Monkey(location=~MATCH.place, holding=MATCH.obj),
        NOT(GoalIsTo(action="walk-to", arguments=MATCH.place)))
    def move_object_to_place(self, place):
        self.declare(GoalIsTo(action="walk-to", arguments=place))

    @Rule(
        AS.goal << GoalIsTo(action="move", arguments__0=MATCH.name, arguments__1=MATCH.place),
        AS.monkey << Monkey(location=MATCH.place, holding=MATCH.obj),
        AS.thing << Thing(name=MATCH.name, weight="light"))
    def drop_object_once_moved(self, name, monkey, thing, goal, place):
        print("Monkey drops the %s." % name)
        self.modify(monkey, holding="blank")
        self.modify(thing, location=place, on_top_of="floor")
        self.retract(goal)

    @Rule(
        AS.goal << GoalIsTo(action="move", arguments__0=MATCH.obj, arguments__1=MATCH.place),
        Thing(name=MATCH.obj, location=MATCH.place))
    def already_moved_object(self, goal):
        self.retract(goal)

class WalkToPlaceRules:
    """WALK TO PLACE RULES"""

    @Rule(
        AS.goal << GoalIsTo(action="walk-to", arguments=MATCH.place),
        Monkey(location=MATCH.place))
    def already_at_place(self, goal):
        self.retract(goal)

    @Rule(
        GoalIsTo(action="walk-to", arguments=MATCH.place),
        Monkey(location=~MATCH.place, on_top_of=NE("floor")),
        NOT(GoalIsTo(action="on", arguments="floor")))
    def get_on_floor_to_walk(self):
        self.declare(GoalIsTo(action="on", arguments="floor"))

    @Rule(
        AS.goal << GoalIsTo(action="walk-to", arguments=MATCH.place),
        AS.monkey << Monkey(location=~MATCH.place, on_top_of="floor", holding="blank"))
    def walk_holding_nothing(self, place, monkey, goal):
        print("Monkey walks to %s." % place)
        self.modify(monkey, location=place)
        self.retract(goal)

    @Rule(
        AS.goal << GoalIsTo(action="walk-to", arguments=MATCH.place),
        AS.monkey << Monkey(location=~MATCH.place, on_top_of="floor", holding=MATCH.obj & NE("blank")))
    def walk_holding_object(self, place, obj, goal, monkey):
        print("Monkey walks to %s holding the %s." % (place, obj))
        self.modify(monkey, location=place)
        self.retract(goal)

class GetOnObjectRules:
    """GET ON OBJECT RULES"""

    @Rule(
        AS.goal << GoalIsTo(action="on", arguments="floor"),
        AS.monkey << Monkey(on_top_of=MATCH.on & NE("floor")))
    def jump_onto_floor(self, on, monkey, goal):
        print("Monkey jumps off the %s onto the floor." % on)
        self.modify(monkey, on_top_of="floor")
        self.retract(goal)

    @Rule(
        GoalIsTo(action="on", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place),
        Monkey(location=~MATCH.place),
        NOT(GoalIsTo(action="walk-to", arguments=MATCH.place)))
    def walk_to_place_to_climb(self, place):
        self.declare(GoalIsTo(action="walk-to", arguments=place))

    @Rule(
        GoalIsTo(action="on", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place),
        Monkey(location=MATCH.place, holding=NE("blank")),
        NOT(GoalIsTo(action="hold", arguments="blank")))
    def drop_to_climb(self):
        self.declare(GoalIsTo(action="hold", arguments="blank"))

    @Rule(
        GoalIsTo(action="on", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of=MATCH.on),
        Monkey(location=MATCH.place, on_top_of=~MATCH.on & ~MATCH.obj, holding="blank"),
        NOT(GoalIsTo(action="on", arguments=MATCH.on)))
    def climb_indirectly(self, on):
        self.declare(GoalIsTo(action="on", arguments=on))

    @Rule(
        AS.goal << GoalIsTo(action="on", arguments=MATCH.obj),
        Thing(name=MATCH.obj, location=MATCH.place, on_top_of=MATCH.on),
        AS.monkey << Monkey(location=MATCH.place, on_top_of=MATCH.on, holding="blank"))
    def climb_directly(self, obj, monkey, goal):
        print("Monkey climbs onto the %s." % obj)
        self.modify(monkey, on_top_of=obj)
        self.retract(goal)

    @Rule(
        AS.goal << GoalIsTo(action="on", arguments=MATCH.obj),
        Monkey(on_top_of=MATCH.obj))
    def already_on_object(self, goal):
        self.retract(goal)

class EatObjectRules:
    """EAT OBJECT RULES"""

    @Rule(
        GoalIsTo(action="eat", arguments=MATCH.obj),
        Monkey(holding=~MATCH.obj),
        NOT(GoalIsTo(action="hold", arguments=MATCH.obj)))
    def hold_to_eat(self, obj):
        self.declare(GoalIsTo(action="hold", arguments=obj))
    
    @Rule(
        AS.goal << GoalIsTo(action="eat", arguments=MATCH.name),
        AS.monkey << Monkey(holding=MATCH.name),
        AS.thing << Thing(name=MATCH.name))
    def satisfy_hunger(self, goal, thing, monkey, name):
        print("Monkey eats the %s." % name)
        self.modify(monkey, holding="blank")
        self.retract(goal)
        self.retract(thing)

class MonkeesAndBananas(ChestUnlockingRules,
                        HoldObjectRules,
                        MoveObjectRules,
                        WalkToPlaceRules,
                        GetOnObjectRules,
                        EatObjectRules,
                        KnowledgeEngine):

    @DefFacts()
    def startup(self):
        """INITIAL STATE"""
        yield Monkey(location="t5-7", on_top_of="green-couch", holding="blank")
        yield Thing(name="green-couch", location="t5-7", on_top_of="floor", weight="heavy")
        yield Thing(name="red-couch", location="t2-2", on_top_of="floor", weight="heavy")
        yield Thing(name="big-pillow", location="t2-2", on_top_of="red-couch", weight="light")
        yield Thing(name="red-chest", location="t2-2", on_top_of="big-pillow", weight="light")
        yield Chest(name="red-chest", contents="ladder", unlocked_by="red-key")
        yield Thing(name="blue-chest", location="t7-7", on_top_of="ceiling", weight="light")
        yield Chest(name="blue-chest", contents="bananas", unlocked_by="blue-key")
        yield Thing(name="blue-couch", location="t8-8", on_top_of="floor", weight="heavy")
        yield Thing(name="green-chest", location="t8-8", on_top_of="ceiling", weight="light")
        yield Chest(name="green-chest", contents="blue-key", unlocked_by="red-key")
        yield Thing(name="red-key", location="t1-3", on_top_of="floor", weight="light")
        yield GoalIsTo(action="eat", arguments="bananas")

In [ ]:
mab = MonkeesAndBananas()
mab.reset()
mab.run()

Monkey jumps off the green-couch onto the floor.
Monkey walks to t2-2.
Monkey climbs onto the red-couch.
Monkey climbs onto the big-pillow.
Monkey grabs the red-chest.
Monkey throws the red-chest off the big-pillow onto the floor.
Monkey jumps off the big-pillow onto the floor.
Monkey walks to t1-3.
Monkey grabs the red-key.
Monkey walks to t2-2 holding the red-key.
Monkey opens the red-chest with the red-key revealing the ladder.
Monkey drops the red-key.
Monkey climbs onto the red-chest.
Monkey grabs the ladder.
Monkey jumps off the red-chest onto the floor.
Monkey walks to t7-7 holding the ladder.
Monkey drops the ladder.
Monkey climbs onto the ladder.
Monkey grabs the blue-chest.
Monkey throws the blue-chest off the ladder onto the floor.
Monkey jumps off the ladder onto the floor.
Monkey grabs the ladder.
Monkey walks to t8-8 holding the ladder.
Monkey drops the ladder.
Monkey climbs onto the ladder.
Monkey grabs the green-chest.
Monkey throws the green-chest off the ladder onto t

## Jogo das Palavras

In [ ]:
from functools import partial, reduce
from itertools import takewhile
import operator as op

from experta import *


class Number(Fact):
    pass


class Letter(Fact):
    pass


class Combination(Fact):
    pass

until_none = partial(takewhile, lambda x: x is not None)


def check_combination(d=None, t=None, l=None, r=None, a=None,
                      e=None, n=None, b=None, o=None, g=None):
    """Check a partial solution."""

    left_1 = list(until_none(reversed([g, e, r, a, l, d])))
    left_2 = list(until_none(reversed([d, o, n, a, l, d])))
    right = list(until_none(reversed([r, o, b, e, r, t])))

    a = sum((l[0] * 10 ** e) + (l[1] * 10 ** e)
            for e, l in enumerate(zip(left_1, left_2)))
    b = sum((l * 10 ** e) for e, l in enumerate(right))

    if len(left_1) < 6:
        a = a % 10**len(right)

    return a == b


def C(name, neq=""):
    """
    Ex: name="c", neq="tar" => W("c") & ~W("t") & ~W("a") & ~W("r")

    """
    if neq:
        return W(name) & reduce(op.and_, [~W(c) for c in neq])
    else:
        return W(name)


class WordGame(KnowledgeEngine):
    @Rule()
    def startup(self):
        print("The problem is")
        print("    GERALD")
        print("  + DONALD")
        print("    ------")
        print("  = ROBERT")
        print()

        for n in range(10):
            self.declare(Number(n))

        for l in set("GERALD" + "DONALD" + "ROBERT"):
            self.declare(Letter(l))

    @Rule(Number(MATCH.n),
          Letter(MATCH.l))
    def generate_combinations(self, n, l):
        self.declare(Combination(l, n))

    @Rule(Combination("D", C("d")),
          Combination("T", C("t", neq="d")),
          TEST(check_combination),
          Combination("L", C("l", neq="dt")),
          Combination("R", C("r", neq="dtl")),
          TEST(check_combination),
          Combination("A", C("a", neq="dtlr")),
          Combination("E", C("e", neq="dtlra")),
          TEST(check_combination),
          Combination("N", C("n", neq="dtlrae")),
          Combination("B", C("b", neq="dtlraen")),
          TEST(check_combination),
          Combination("O", C("o", neq="dtlraenb")),
          Combination("G", C("g", neq="dtlraenbo")),
          TEST(check_combination))
    def find_solution(self, g, e, r, a, l, d, o, n, b, t):
        print("A solution is:")
        print("  G =", g)
        print("  E =", e)
        print("  R =", r)
        print("  A =", a)
        print("  L =", l)
        print("  D =", d)
        print("  O =", o)
        print("  N =", n)
        print("  B =", b)
        print("  T =", t)
        print()
        print("    ", g, e, r, a, l, d)
        print("  + ", d, o, n, a, l, d)
        print("    ", "------")
        print("  = ", r, o, b, e, r, t)
        print()

In [ ]:
wg = WordGame()
wg.reset()
wg.run()

The problem is
    GERALD
  + DONALD
    ------
  = ROBERT

A solution is:
  G = 1
  E = 9
  R = 7
  A = 4
  L = 8
  D = 5
  O = 2
  N = 6
  B = 3
  T = 0

     1 9 7 4 8 5
  +  5 2 6 4 8 5
     ------
  =  7 2 3 9 7 0



## Pedra, Papel e Tesoura

In [ ]:
from experta import *
from experta.fact import *
import random

NERD = True

class WinTotals(Fact):
    human = Field(int, default=0)
    computer = Field(int, default=0)
    ties = Field(int, default=0)


class Results(Fact):
    winner = Field(str, mandatory=True)
    loser = Field(str, mandatory=True)
    why = Field(str, mandatory=True)


class ValidAnswer(Fact):
    answer = Field(str, mandatory=True)
    key = Field(str, mandatory=True)


class Action(Fact):
    pass


class HumanChoice(Fact):
    pass


class ComputerChoice(Fact):
    pass


class RockPaperScissors(KnowledgeEngine):
    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')
    
    @DefFacts()
    def game_rules(self, is_nerd=False):
        """Declare game rules and valid input keys for the user."""
        self.valid_answers = dict()
        
        yield Results(winner='rock', loser='scissors', why='Rock smashes scissors')
        yield Results(winner='paper', loser='rock', why='Paper covers rock')
        yield Results(winner='scissors', loser='paper', why='Scissors cut paper')
        yield ValidAnswer(answer='rock', key='r')
        yield ValidAnswer(answer='paper', key='p')
        yield ValidAnswer(answer='scissors', key='s')

        if is_nerd:
            yield Results(winner='rock', loser='lizard', why='Rock crushes lizard')
            yield Results(winner='spock', loser='rock', why='Spock vaporizes rock')
            yield Results(winner='spock', loser='scissors', why='Spock smashes scissors')
            yield Results(winner='paper', loser='spock', why='Paper disproves Spock')
            yield Results(winner='scissors', loser='lizard', why='Scissors decapitates lizard')
            yield Results(winner='lizard', loser='paper', why='Lizard eats paper')
            yield Results(winner='lizard', loser='spock', why='Lizard poisons Spock')
            yield ValidAnswer(answer='spock', key='k')
            yield ValidAnswer(answer='lizard', key='l')
    
    @Rule()
    def startup(self):
        print("Lets play a game!")
        print("You choose rock, paper, or scissors,")
        print("and I'll do the same.")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))
    
    @Rule(NOT(Action()),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.key))
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer

    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        self.declare(HumanChoice(res))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.choice),
          AS.f2 << Action('get-human-move'))
    def good_human_move(self, f1, f2, answer):
        self.retract(f1)
        self.retract(f2)
        self.declare(HumanChoice(answer))
        self.declare(Action('get-computer-move'))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          NOT(ValidAnswer(key=MATCH.choice)),
          AS.f2 << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print("Sorry %s is not a valid answer" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))
    
    #
    # COMPUTER MOVE RULES
    #
    @Rule(AS.f1 << Action('get-computer-move'))
    def get_computer_move(self, f1):
        choice = random.choice(list(self.valid_answers.values()))
        self.retract(f1)
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))

    #
    # WIN DETERMINATION RULES
    #
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.w << WinTotals(computer=MATCH.cw),
          Results(winner=MATCH.cc,
                  loser=MATCH.hc,
                  why=MATCH.explanation))
    def computer_wins(self, f1, f2, f3, w, cw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, computer=cw + 1)
        print("Computer wins!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          'w' << WinTotals(human=MATCH.hw),
          Results(winner=MATCH.hc,
                  loser=MATCH.cc,
                  why=MATCH.explanation))
    def humans_wins(self, f1, f2, f3, w, hw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, human=hw + 1)
        print("You win!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.cc),
          AS.w << WinTotals(ties=MATCH.nt))
    def tie(self, f1, f2, f3, w, nt):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, ties=nt + 1)
        print("Tie! Ha-ha!")
        self.declare(Action('determine-play-again'))
    
    #
    # PLAY AGAIN RULE
    #
    @Rule(AS.f1 << Action('determine-play-again'),
          WinTotals(computer=MATCH.ct,
                    human=MATCH.ht,
                    ties=MATCH.tt))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1)
        if not self.yes_or_no("Play again?"):
            print("You won", ht, "game(s).")
            print("Computer won", ct, "game(s).")
            print("We tied", tt, "game(s).")
            self.halt()
        else:
            self.declare(Action('get-human-move'))

In [ ]:
rps = RockPaperScissors()
rps.reset()
rps.run()

In [ ]:
rps = RockPaperScissors()
rps.reset(is_nerd=True)
rps.run()